In [7]:
import os
import tempfile
from pathlib import Path

from rich import print as rprint
from utils import create_mock_circuit_dir

from entitysdk import Client, ProjectContext, models, types

entitycore_api_url = "http://127.0.0.1:8000"
project_context = ProjectContext(
    virtual_lab_id="a98b7abc-fc46-4700-9e3d-37137812c730",
    project_id="0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6",
)
token = os.getenv("ACCESS_TOKEN", "XXX")
client = Client(api_url=entitycore_api_url, project_context=project_context, token_manager=token)

## Create a subject

In [2]:
species = client.search_entity(
    entity_type=models.Species, query={"name__ilike": "Mus musculus"}
).first()

subject = models.Subject(
    name="my-subject", description="my-subject-description", sex="male", species=species
)

subject = client.register_entity(subject)

## Create circuit model

In [3]:
brain_region = client.search_entity(entity_type=models.BrainRegion, query={"acronym": "CB"}).first()


circuit = models.Circuit(
    name="my-circuit",
    description="my-circuit",
    subject=subject,
    brain_region=brain_region,
    number_synapses=2,
    number_neurons=5,
    number_connections=10,
    scale="microcircuit",
    build_category="em_reconstruction",
)

## Register circuit

In [4]:
registered_circuit = client.register_entity(circuit)

In [5]:
rprint(registered_circuit)

Circuit(
    assets=[],
    id=UUID('7e87955f-e0d9-40d7-aaed-528b5bca986c'),
    update_date=datetime.datetime(2025, 9, 19, 14, 3, 25, 268988, tzinfo=TzInfo(UTC)),
    creation_date=datetime.datetime(2025, 9, 19, 14, 3, 25, 268988, tzinfo=TzInfo(UTC)),
    name='my-circuit',
    description='my-circuit',
    type=<EntityType.circuit: 'circuit'>,
    created_by=Person(
        id=UUID('ab00247a-775d-4367-b122-693edc2aecbe'),
        update_date=None,
        creation_date=None,
        type='person',
        pref_label='Admin User',
        given_name=None,
        family_name=None
    ),
    updated_by=Person(
        id=UUID('ab00247a-775d-4367-b122-693edc2aecbe'),
        update_date=None,
        creation_date=None,
        type='person',
        pref_label='Admin User',
        given_name=None,
        family_name=None
    ),
    authorized_public=False,
    authorized_project_id=UUID('0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6'),
    contributions=[],
    legacy_id=None,
    experiment_date=None,
    contact_email=None,
    subject=Subject(
        assets=[],
        id=UUID('831e1c2d-f6f9-4ff5-bcb1-4ff07ad2b0c0'),
        update_date=None,
        creation_date=None,
        name='my-subject',
        description='my-subject-description',
        type=None,
        created_by=None,
        updated_by=None,
        authorized_public=False,
        authorized_project_id=None,
        contributions=None,
        legacy_id=None,
        sex=<Sex.male: 'male'>,
        weight=None,
        age_value=None,
        age_min=None,
        age_max=None,
        age_period=None,
        species=Species(
            id=UUID('b7ad4cca-4ac2-4095-9781-37fb68fe9ca1'),
            update_date=None,
            creation_date=None,
            name='Mus musculus',
            taxonomy_id='NCBITaxon:10090'
        ),
        strain=None
    ),
    brain_region=BrainRegion(
        id=UUID('062ebc92-cebb-498a-a76d-b5ef1baf02cf'),
        update_date=datetime.datetime(2025, 6, 27, 11, 10, 27, 186088, tzinfo=TzInfo(UTC)),
        creation_date=datetime.datetime(2025, 6, 27, 11, 10, 27, 186088, tzinfo=TzInfo(UTC)),
        name='Cerebellum',
        annotation_value=512,
        acronym='CB',
        parent_structure_id=UUID('4642cddb-4fbe-4aae-bbf7-0946d6ada066'),
        hierarchy_id=UUID('e3e70682-c209-4cac-a29f-6fbed82c07cd'),
        color_hex_triplet='F0F080'
    ),
    license=None,
    published_in=None,
    has_morphologies=False,
    has_point_neurons=False,
    has_electrical_cell_models=False,
    has_spines=False,
    number_neurons=5,
    number_synapses=2,
    number_connections=10,
    scale=<CircuitScale.microcircuit: 'microcircuit'>,
    build_category=<CircuitBuildCategory.em_reconstruction: 'em_reconstruction'>,
    root_circuit_id=None,
    atlas_id=None
)

## Upload directory

In [8]:
with tempfile.TemporaryDirectory() as tdir:
    # create a hierarchy of files to upload
    create_mock_circuit_dir(tdir)

    files = {str(path.relative_to(tdir)): path for path in Path(tdir).rglob("*") if path.is_file()}

    directory_asset = client.upload_directory(
        entity_id=registered_circuit.id,
        entity_type=models.Circuit,
        name="circuit",
        label=types.AssetLabel.sonata_circuit,
        paths=files,
    )

## List directory

In [9]:
files = client.list_directory(
    entity_id=registered_circuit.id, entity_type=models.Circuit, asset_id=directory_asset.id
)

In [10]:
rprint(files)

DetailedFileList(
    files={
        PosixPath('circuit_config.json'): DetailedFile(
            name='circuit_config.json',
            size=0,
            last_modified=datetime.datetime(2025, 9, 19, 14, 5, 2, 174000, tzinfo=TzInfo(UTC))
        ),
        PosixPath('sonata/edges.h5'): DetailedFile(
            name='sonata/edges.h5',
            size=0,
            last_modified=datetime.datetime(2025, 9, 19, 14, 5, 2, 178000, tzinfo=TzInfo(UTC))
        ),
        PosixPath('sonata/nodes.h5'): DetailedFile(
            name='sonata/nodes.h5',
            size=0,
            last_modified=datetime.datetime(2025, 9, 19, 14, 5, 2, 181000, tzinfo=TzInfo(UTC))
        )
    }
)

## Download directory

In [11]:
with tempfile.TemporaryDirectory() as tdir:
    client.download_directory(
        entity_id=registered_circuit.id,
        entity_type=models.Circuit,
        asset_id=directory_asset.id,
        output_path=Path(tdir),
    )
    files = {str(path.relative_to(tdir)): path for path in Path(tdir).rglob("*") if path.is_file()}
    rprint(files)

{
    'circuit/circuit_config.json': 
PosixPath('/var/folders/rf/qpcjsbls70n_vrc8vykk98_h0000gn/T/tmpmao71e3e/circuit/circuit_config.json'),
    'circuit/sonata/edges.h5': 
PosixPath('/var/folders/rf/qpcjsbls70n_vrc8vykk98_h0000gn/T/tmpmao71e3e/circuit/sonata/edges.h5'),
    'circuit/sonata/nodes.h5': 
PosixPath('/var/folders/rf/qpcjsbls70n_vrc8vykk98_h0000gn/T/tmpmao71e3e/circuit/sonata/nodes.h5')
}

## Fetch Circuit

In [12]:
fetched = client.get_entity(entity_id=registered_circuit.id, entity_type=models.Circuit)

In [13]:
rprint(fetched)

Circuit(
    assets=[
        Asset(
            id=UUID('2d376836-919e-4118-a0bc-a28f506248e6'),
            update_date=None,
            creation_date=None,
            path='circuit',
            full_path='private/a98b7abc-fc46-4700-9e3d-37137812c730/0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6/assets/cir
cuit/7e87955f-e0d9-40d7-aaed-528b5bca986c/circuit',
            is_directory=True,
            content_type=<ContentType.application_vnd_directory: 'application/vnd.directory'>,
            size=-1,
            sha256_digest=None,
            status=<AssetStatus.created: 'created'>,
            meta={},
            label=<AssetLabel.sonata_circuit: 'sonata_circuit'>
        )
    ],
    id=UUID('7e87955f-e0d9-40d7-aaed-528b5bca986c'),
    update_date=datetime.datetime(2025, 9, 19, 14, 3, 25, 268988, tzinfo=TzInfo(UTC)),
    creation_date=datetime.datetime(2025, 9, 19, 14, 3, 25, 268988, tzinfo=TzInfo(UTC)),
    name='my-circuit',
    description='my-circuit',
    type=<EntityType.circuit: 'circuit'>,
    created_by=Person(
        id=UUID('ab00247a-775d-4367-b122-693edc2aecbe'),
        update_date=None,
        creation_date=None,
        type='person',
        pref_label='Admin User',
        given_name=None,
        family_name=None
    ),
    updated_by=Person(
        id=UUID('ab00247a-775d-4367-b122-693edc2aecbe'),
        update_date=None,
        creation_date=None,
        type='person',
        pref_label='Admin User',
        given_name=None,
        family_name=None
    ),
    authorized_public=False,
    authorized_project_id=UUID('0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6'),
    contributions=[],
    legacy_id=None,
    experiment_date=None,
    contact_email=None,
    subject=Subject(
        assets=[],
        id=UUID('831e1c2d-f6f9-4ff5-bcb1-4ff07ad2b0c0'),
        update_date=None,
        creation_date=None,
        name='my-subject',
        description='my-subject-description',
        type=None,
        created_by=None,
        updated_by=None,
        authorized_public=False,
        authorized_project_id=None,
        contributions=None,
        legacy_id=None,
        sex=<Sex.male: 'male'>,
        weight=None,
        age_value=None,
        age_min=None,
        age_max=None,
        age_period=None,
        species=Species(
            id=UUID('b7ad4cca-4ac2-4095-9781-37fb68fe9ca1'),
            update_date=None,
            creation_date=None,
            name='Mus musculus',
            taxonomy_id='NCBITaxon:10090'
        ),
        strain=None
    ),
    brain_region=BrainRegion(
        id=UUID('062ebc92-cebb-498a-a76d-b5ef1baf02cf'),
        update_date=datetime.datetime(2025, 6, 27, 11, 10, 27, 186088, tzinfo=TzInfo(UTC)),
        creation_date=datetime.datetime(2025, 6, 27, 11, 10, 27, 186088, tzinfo=TzInfo(UTC)),
        name='Cerebellum',
        annotation_value=512,
        acronym='CB',
        parent_structure_id=UUID('4642cddb-4fbe-4aae-bbf7-0946d6ada066'),
        hierarchy_id=UUID('e3e70682-c209-4cac-a29f-6fbed82c07cd'),
        color_hex_triplet='F0F080'
    ),
    license=None,
    published_in=None,
    has_morphologies=False,
    has_point_neurons=False,
    has_electrical_cell_models=False,
    has_spines=False,
    number_neurons=5,
    number_synapses=2,
    number_connections=10,
    scale=<CircuitScale.microcircuit: 'microcircuit'>,
    build_category=<CircuitBuildCategory.em_reconstruction: 'em_reconstruction'>,
    root_circuit_id=None,
    atlas_id=None
)